Lending Club，P2P平台，公司位于美国旧金山。2014年12月12日开始在纽交所挂牌交易，代码LC。目前总市值54亿美元。

所用数据是 LoanStats_2016Q4.csv 和LoanStats_2017Q1.csv ，在 LoanStats_2016Q4.csv 上训练模型和验证模型，在LoanStats_2017Q1.csv上加载模型用来测试模型表现。

所用模型是LogisticRegression 和 XGBClassifier，默认参数未调参

数据来源：https://www.lendingclub.com/info/download-data.action
特征变量解释：http://www.docin.com/p-635342290.html
目标变量解释：http://www.doc88.com/p-5089631302589.html

# Step 1: 数据加载


In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
data=pd.read_csv('LoanStats_2016Q4.csv',skiprows=1)
data.head()
data0=data
#import missingno
#missingno.matrix(data)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


我们发现共103548个样本，145个特征。

缺失值可视化发现，很多特征含有缺失值。

# Step 2: 数据预处理与特征过滤

In [2]:
check_null=data.isnull().sum(axis=0).sort_values(ascending=False)/float(len(data))
check_null[check_null>0]

sec_app_mths_since_last_major_derog           1.000000
member_id                                     1.000000
url                                           1.000000
revol_bal_joint                               1.000000
sec_app_earliest_cr_line                      1.000000
sec_app_inq_last_6mths                        1.000000
sec_app_mort_acc                              1.000000
sec_app_open_acc                              1.000000
sec_app_revol_util                            1.000000
sec_app_open_act_il                           1.000000
sec_app_num_rev_accts                         1.000000
sec_app_chargeoff_within_12_mths              1.000000
sec_app_collections_12_mths_ex_med            1.000000
desc                                          0.999990
id                                            0.999981
settlement_percentage                         0.997943
settlement_term                               0.997943
settlement_date                               0.997943
debt_settl

预处理：删除缺失比例0.5以上的变量、常数变量、检查有没有重复变量

In [3]:
data=data.dropna(thresh=len(data)*0.5,axis=1)
data=data.loc[:,data.apply(pd.Series.nunique)!=1]
len(set(list(data.columns)))==data.shape[1]
data.shape

(103548, 102)

再继续看看下数据集形状，发现特征从145变成了102

数据集中很多特征是关于投资人的数据，与建模无关，所以我需要过滤变量；同时为了便于大家理解，我把特征重命名为中文。


In [18]:
print(','.join(["'{}'".format(s) for s in data.columns]))

'loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade','emp_title','emp_length','home_ownership','annual_inc','verification_status','issue_d','loan_status','pymnt_plan','purpose','title','zip_code','addr_state','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','mths_since_last_delinq','open_acc','pub_rec','revol_bal','revol_util','total_acc','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d','last_credit_pull_d','collections_12_mths_ex_med','application_type','acc_now_delinq','tot_coll_amt','tot_cur_bal','open_acc_6m','open_act_il','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_u

In [4]:
col=['loan_amnt','term','int_rate','installment','grade','sub_grade','emp_length','home_ownership','annual_inc','verification_status',\
     'issue_d','loan_status','purpose','addr_state','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_bal',\
     'revol_util','total_acc','acc_now_delinq','delinq_amnt','pub_rec_bankruptcies','tax_liens']
data=data[col]
new_col={
'loan_amnt':'贷款金额',
'term':'贷款期限',
'int_rate':'利率',
'installment':'每月还款金额',
'grade':'贷款等级',
'sub_grade':'基础等级',
'emp_length':'工作年限',#（0：少于1年，10：10年及以上）
'home_ownership':'房屋所有权',#(出租、自有、按揭、其他)
'annual_inc':'年收入',
'verification_status':'收入是否由LC验证',
'issue_d':'放款日期',
'loan_status':'target',
'purpose':'贷款目的',
'addr_state':'申请人所在洲',
'dti':'月负债比',
'delinq_2yrs':'过去两年借款人逾期30天以上的数字',
'earliest_cr_line':'信用报告最早日期',
'inq_last_6mths':'过去6个月内被查询次数',
'open_acc':'未还清贷款额度',
'pub_rec':'摧毁公共记录的数量',
'revol_bal':'总贷款金额',
'revol_util':'额度循环使用率',
'total_acc':'总贷款笔数',
'acc_now_delinq':'拖欠的账户数量',
'delinq_amnt':'拖欠的逾期款项',
'pub_rec_bankruptcies':'公开记录破产的数量',
'tax_liens':'留置税数量'}

data=data.rename(columns=new_col)


In [25]:
for c in data.columns:
    print(c,data[c].dropna().count())

贷款金额 103548
贷款期限 103548
利率 103548
每月还款金额 103548
贷款等级 103548
基础等级 103548
工作年限 103548
房屋所有权 103548
年收入 103548
收入是否由LC验证 103548
放款日期 103548
target 103548
贷款目的 103548
申请人所在洲 103548
月负债比 103548
过去两年借款人逾期30天以上的数字 103548
信用报告最早日期 103548
过去6个月内被查询次数 103548
未还清贷款额度 103548
摧毁公共记录的数量 103548
总贷款金额 103548
额度循环使用率 103548
总贷款笔数 103548
拖欠的账户数量 103548
拖欠的逾期款项 103548
公开记录破产的数量 103548
留置税数量 103548


In [5]:
from imp import reload
import reportgen as rpt
reload(rpt)
rpt.AnalysisReport(data,'club report')

ValueError: Cannot convert non-finite values (NA or inf) to integer

# Step 3: 特征抽象与可视化

我先把变量分为连续变量与分类变量，发现有11个连续变量，15个分类变量


In [ ]:
len(continuous_var)
len(categorical_var)

In [ ]:
col='target'
print(data[col].value_counts())

发现目标变量有点乱，没关系。

特征抽象：把目标变量各水平转为我们所需要的0、1形式。

处理后发现正常客户17731，违约客户6816

特征中有时间变量：信用报告最早日期、放款日期。



我们知道美国的征信数据是比较完善的，那我想知道会不会开启“信用报告天赋”越早，表现越好呢？

这里稍微做了个特征衍生，创建了新变量days，含义是放款时，客户拥有信用报告的天数

In [ ]:
data['t1_new']=data['信用报告最早日期'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m'))
data['t2_new']=data['放款日期'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m'))
data['days']=daa[['t1_new','t2_new']].apply(lambda x:(x[1]-x[0]).days,axis=1)
data['days'].head()

在做过分类变量水平合并和删除严重不平衡的分类变量后，接下来就是可视化了：让我们看一下各变量的分布以及与目标变量的关系吧

有一句话叫：老板看不懂代码，但能看得懂图。get可视化技能很重要哦！



对于以上图片相信大家都能看懂，我就不赘述了，接下来再次预览数据，还剩下22个特征

In [ ]:
data['string_var'].head()


# Step 4: 数据补缺



我们先看一下处理后的数据缺失情况吧

In [ ]:
data.isnull().sum(axis=0).sort_values(ascending=False)/float(len(data))

在这里，我把连续变量用随机抽样补缺，分类变量用众数补缺，同时统一字符变量为大写

In [ ]:
for col in string_var:
    data[col]=data[col].apply(lambda x:str(x).upper)
    

# Step 5: 卡方分箱，计算WOE和IV值



为了创建申请评分卡，我们不用One-hot、不用归一化，而是选择卡方分箱和WOE编码。

这里先解释下卡方分箱、WOE和IV的概念：

![](641.webp)
![](640.webp)


卡方分箱和WOE计算过程较为繁琐，这里不再赘述。最终保存如下数据：



In [ ]:
import pickle
f=open('var_WOE.pkl','wb')
pickle.dump(var_WOE,f)
f.close()

f=open('var_IV.pkl','wb')
pickle.dump(var_IV,f)
f.close()

f=open('var_cutoff.pkl','wb')
pickle.dump(var_cutoff,f)
f.close()

f=open('encoded_features.pkl','wb')
pickle.dump(encoded_features,f)
f.close()

f=open('merged_features.pkl','wb')
pickle.dump(merged_features,f)
f.close()


虽然分箱前只有22个特征，但还是要筛选变量的，经过IV筛选和相关系数筛选后，


In [12]:
'''单变量分析，找出IV大于0.02的变量'''
iv_threshould=0.02
varByIV=[k for k,v in var_IV.items() if v > iv_threshoyld]
print('剩余%d个变量'%len(varByIV))
print(varByIV)
'''多变量分析，保留相关性大于阈值0.6的变量'''
var_IV_selected={k:var_IV[k] for k in varByIV}
var_IV_sorted=sorted(var_IV_selected.items())
var_IV_sorted=[i[0] for i in var_IV_sorted] #IV排序的列名称
removed_var=[] #两两相关性判断，被删除的变量
roh_thresould = 0.6

for i in range(len(var_IV_sorted)-1):
    if var_IV_sorted[i] not in removed_var:
        x1=var_IV_sorted[i]+'_WOE'
        for j in range(i+1,len(var_IV_sorted)):
            if var_IV_sorted[j] not in removed_var:
                x2=var_IV_sorted[j] + '_WOE'
                roh=np.corrcoef([data[x1],data[x2]])[0,1]
                if abs(roh) >= roh_thresould:
                    print('{} 和 {} 的相关系数是 {}'.format(x1,x2,str(roh)))
                    if var_IV[var_IV_sorted[i]] > var_IV[var_IV_sorted[j]]: #删除IV值较小的变量
                        removed_var.append(var_IV_sorted[j])
                    else:
                        removed_var.append(var_IV_sorted[i])
                        
var_IV_sorted_2=[i for i in var_IV_sorted if i not in removed_var]
print('剩余',len(var_IV_sorted_2),'个变量')
print(var_IV_sorted_2)

SyntaxError: invalid syntax (<ipython-input-12-f05677cd3877>, line 24)

2016Q4数据中，逾期坏样本比例28%，我们可以不用"过抽样"。本着探讨的精神， 我会用逻辑回归，XGBOOST 这2个模型，未SMOT抽样，SMOTE抽样这2种情况，共训练4个模型；


之后这4个模型会在在2017Q1数据中分别加载，看看效果。


先看smote抽样方法：

In [ ]:
X=data[var_WOE_list]
y=data['target']

In [ ]:
print('采样前')
n_sample = y.shape[0]
n_pos_sample = y[y==0].shape[0]
n_neg_sample = y[y==1].shape[0]
print('特征个数：',X.shape[1])
print()
print('样本个数{}，正常样本{},逾期样本{},逾期样本占{:.0%}'.format(n_sample,n_pos_sample,n_neg_sample,n_neg_sample/n_sample))


In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=1)
X,y=smote.fit_sample(X,y)

In [ ]:
print('采样后')
n_sample = y.shape[0]
n_pos_sample = y[y==0].shape[0]
n_neg_sample = y[y==1].shape[0]
print('特征个数：',X.shape[1])
print()
print('样本个数{}，正常样本{},逾期样本{},逾期样本占{:.0%}'.format(n_sample,n_pos_sample,n_neg_sample,n_neg_sample/n_sample))

逻辑回归有个显著性检验，这一步也可以筛选变量：


summary这几个变量都是显著的，下面建模



第一个模型是经过smote抽样后的逻辑回归模型：

In [ ]:
lr=LR()
lr.fit(X_train,y_train)
y_train_label=lr.predict(X_train)
y_test_label=lr.predict(X_test)

from sklearn.metrics import accuracy_score
print('训练集准确率：{:.2%}'.format(accuracy_score(y_train_label,y_train)))
print('测试集准确率：{:.2%}'.format(accuracy_score(y_test_label,y_test)))

我们发现统计指标不是很好，别灰心，惊喜在下面！

再看看经过smote抽样的xgboost模型：

是不是很激动，统计指标辣么高！

再看更鸡冻的：红线代表逾期客户的分数分布，蓝线代表正常客户的分数分布。



我们发现：哇塞，低分与高分分的好开，直接拒绝低分的，放过高分的，人工审核中间分的，太棒了！

但是，它过拟合了。接下来告诉你什么



我们再看看未经过smote抽样的数据，分别用逻辑回归和xgboost试试。

先看LR：

In [ ]:
'''逻辑回归'''
from sklearn.cross_validation import train_test_split as sp
from sklearn.linear_model import LogisticRegression as LR

X=data[features_selection]
y=data['traget']
X_train,X_test,y_train,y_test=sp(X,y,test_size=0.3,random_stated=1)

lr=LR()
lr.fit(X_train,y_train)
y_train_label=lr.predict(X_train)
y_test_label=lr.predict(X_test)

from sklearn.metrics import accuracy_score
print('训练集准确率：{:.2%}'.format(accuracy_score(y_train_label,y_train)))
print('测试集准确率：{:.2%}'.format(accuracy_score(y_test_label,y_test)))

我们会发现，未经过smote抽样的逻辑回归准确率大幅提升，KS和AUC变化不明显。



再看XGBOOST：

In [ ]:
from sklearn.cross_validation import train_test_split as sp
X_train,X_test,y_train,y_test=sp(X,y,test_size=0.3,random_stated=1)

modelfit(xgb1,X_train,y_train)

统计指标大幅下降！  但是比逻辑回归略好



这4个模型训练好了，我们看下这4个模型在2017年第1季度数据上的表现



先加载未经过smote采样的逻辑回归模型

可见：虽然说是不同的数据，但逻辑回归表现的还是比较稳定的



再加载经过smote采样的逻辑回归模型我们发现smote采样后的模型，测试准确率低了很多。



前辈经验是10%坏样本以上的数据尽量先不用smote采样哦



同样的方法加载XGBOOST模型，这里就不赘述了。



最后把汇总的统计指标发给大家，怎么分析，仁者见仁智者见智啦。



X轴：0表示未smote抽样，load表示加载模型

Y轴：统计指标实际值

我们发现smote采样后的模型，测试准确率低了很多。



前辈经验是10%坏样本以上的数据尽量先不用smote采样哦



同样的方法加载XGBOOST模型，这里就不赘述了。



最后把汇总的统计指标发给大家，怎么分析，仁者见仁智者见智啦。



X轴：0表示未smote抽样，load表示加载模型

Y轴：统计指标实际值